#  EQUAL WEIGHT S&P 500 Index Fund

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

    
    

# Importing Out List of Stocks

In [2]:
stocks = pd.read_csv('C:/Users/sidka/Downloads/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


# Acquiring an API token

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

# Making Our first API Call
### We need The following from API Calls
 Market Cap for each stock
 
 Price for each stock

In [4]:
symbol = 'AAPL'
api_url = f"https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
data.keys()

dict_keys(['avgTotalVolume', 'calculationPrice', 'change', 'changePercent', 'close', 'closeSource', 'closeTime', 'companyName', 'currency', 'delayedPrice', 'delayedPriceTime', 'extendedChange', 'extendedChangePercent', 'extendedPrice', 'extendedPriceTime', 'high', 'highSource', 'highTime', 'iexAskPrice', 'iexAskSize', 'iexBidPrice', 'iexBidSize', 'iexClose', 'iexCloseTime', 'iexLastUpdated', 'iexMarketPercent', 'iexOpen', 'iexOpenTime', 'iexRealtimePrice', 'iexRealtimeSize', 'iexVolume', 'lastTradeTime', 'latestPrice', 'latestSource', 'latestTime', 'latestUpdate', 'latestVolume', 'low', 'lowSource', 'lowTime', 'marketCap', 'oddLotDelayedPrice', 'oddLotDelayedPriceTime', 'open', 'openTime', 'openSource', 'peRatio', 'previousClose', 'previousVolume', 'primaryExchange', 'symbol', 'volume', 'week52High', 'week52Low', 'ytdChange', 'isUSMarketOpen'])

In [ ]:
columns = ['Ticker', 'Price', 'Market Capitalization', 'Number of Shares to Buy']
new_df = pd.DataFrame(columns = columns)



def add_cols(item):
    symbol = item['Ticker']
    num = 0
    try:
        api_url = api_url = f"https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
        data = requests.get(api_url).json()
        item['Price'] = data['latestPrice']
        item['Market Capitalization'] = data['marketCap']
        item['Number of Shares to Buy'] = "N/A"
        return item
    except:
        num += 1
    
    
df = stocks.apply(add_cols, axis=1)    

# USING Batch API Call

In [ ]:
df.head()

In [ ]:
# Function to split a list into chunks of size n
def chunks_split(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i + size]

In [ ]:
grps = list(chunks_split(stocks['Ticker'].values, 100))

In [ ]:
comma_sep_grps = []
for i in range(0, len(grps)):
    comma_sep_grps.append(','.join(grps[i]))

In [ ]:
final_dataframe = pd.DataFrame(columns=columns)
final_dataframe

In [ ]:
lst_series = []
for comma_sep_grp in comma_sep_grps:
    batch_api_request = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={comma_sep_grp}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_request).json()
    for symbol in comma_sep_grp.split(','):
        try:
            temp_series = pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ], index=columns)
            lst_series.append(temp_series)
        except:
            pass
        
final_dataframe = pd.DataFrame(lst_series)

In [ ]:
final_dataframe

# Calculating the number of shares to buy

In [ ]:
portfolio_size = input("Enter the size of your portfolio: ")
try:
    value = float(portfolio_size)
except TypeError:
    print("Please Enter a number/float value!")
    portfolio_size = input("Enter the size of your portfolio: ")
    value = float(portfolio_size)

In [ ]:
positional_size = value/len(final_dataframe.index)

def num_shares_to_buy(x):
    x['Number of Shares to Buy'] = math.floor(positional_size / x['Price'])
    return x

final_dataframe.transform(num_shares_to_buy, axis=1)

# Using xlsxwriter 

In [ ]:
writer = pd.ExcelWriter("recommended trades.xlsx", engine='xlsxwriter')

In [ ]:
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
{
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

dollar_format = writer.book.add_format(
{
    'num_format': '$0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})
integer_format = writer.book.add_format(
{
    'num_format': '0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})


In [ ]:
columns_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
    
}

In [ ]:
for column in columns_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, columns_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', columns_formats[column][0], string_format)

# Saving Excel output

In [ ]:
writer.save()